In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.mllib.fpm import FPGrowth
import re
from random import random
from pyspark import SparkContext

In [2]:
import os
config = pyspark.SparkConf().set('spark.executor.memory', '2GB')\
         .set('spark.cores.max', '2')\
         .set('spark.port.maxRetries','200')\
         .set('spark.ui.port', 4040 + hash(os.getcwd().split("/")[2])%200 )    
sc = pyspark.SparkContext(appName="Professional", master='spark://polyp1:7077',conf = config )

ss = pyspark.sql.SparkSession.builder.config(conf = config).getOrCreate()

In [3]:
#### Input data
sc = ss.sparkContext
taxiRDD = sc.textFile("/scratch/ISE495/taxi_data/green_tripdata_2015-09.csv")


In [11]:
! head /scratch/ISE495/taxi_data/green_tripdata_2015-09.csv

In [7]:
print taxiRDD.take(1)
print taxiRDD.count()

[u'2,2015-09-01 00:02:34,2015-09-01 00:02:38,N,5,-73.979484558105469,40.684955596923828,-73.97943115234375,40.685020446777344,1,.00,7.8,0,0,1.95,0,,0,9.75,1,2']
1494926


In [10]:
taxiC = taxiRDD.map(lambda s: s.split(","))
print taxiC.take(1)

city1 = taxiC.take(1000)

def city(s):
    cit = []
    word = s.split(",")
    if (1.323942*word[5] + 138.669195) > word[6]:
        word[5] = 'NJ'
    else:
        word[5] = 'NY'
    if (1.323942*word[7] + 138.669195) > word[8]:
        word[7] = 'NJ'
    else:
        word[7] = 'NY'
        
    return word

  

print taxiC.map(city).take(1)


[[u'2', u'2015-09-01 00:02:34', u'2015-09-01 00:02:38', u'N', u'5', u'-73.979484558105469', u'40.684955596923828', u'-73.97943115234375', u'40.685020446777344', u'1', u'.00', u'7.8', u'0', u'0', u'1.95', u'0', u'', u'0', u'9.75', u'1', u'2']]


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 4 times, most recent failure: Lost task 0.3 in stage 10.0 (TID 26, 10.0.0.30, executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/opt/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark-2.2.0-bin-hadoop2.7/python/pyspark/rdd.py", line 1339, in takeUpToNumLeft
    yield next(iterator)
  File "<ipython-input-10-bde8913ab554>", line 8, in city
AttributeError: 'l

In [4]:
from pyspark.sql import Row
taxiC = taxiRDD.map(lambda s: s.split(","))

taxiDF = taxiC.toDF(["VendorID","lpep_pickup_datetime","Lpep_dropoff_datetime","Store_and_fwd_flag","RateCodeID","Pickup_longitude","Pickup_latitude","Dropoff_longitude","Dropoff_latitude","Passenger_count","Trip_distance","Fare_amount","Extra","MTA_tax","Tip_amount","Tolls_amount","Ehail_fee","improvement_surcharge","Total_amount","Payment_type","Trip_type"])



In [21]:
print taxiDF.show()

Py4JJavaError: An error occurred while calling o167.showString.
: java.lang.IllegalStateException: SparkContext has been shutdown
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2014)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2043)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2062)
	at org.apache.spark.sql.execution.SparkPlan.executeTake(SparkPlan.scala:336)
	at org.apache.spark.sql.execution.CollectLimitExec.executeCollect(limit.scala:38)
	at org.apache.spark.sql.Dataset.org$apache$spark$sql$Dataset$$collectFromPlan(Dataset.scala:2853)
	at org.apache.spark.sql.Dataset$$anonfun$head$1.apply(Dataset.scala:2153)
	at org.apache.spark.sql.Dataset$$anonfun$head$1.apply(Dataset.scala:2153)
	at org.apache.spark.sql.Dataset$$anonfun$55.apply(Dataset.scala:2837)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:65)
	at org.apache.spark.sql.Dataset.withAction(Dataset.scala:2836)
	at org.apache.spark.sql.Dataset.hea

In [5]:
taxiDF.createOrReplaceTempView("part")
#ss.sql("SELECT *, IF(1.323942*FLOAT(part.Pickup_longitude) + 138.669195 < FLOAT(part.Pickup_latitude),'NJ','NY') as pickup_state, IF(1.323942*FLOAT(part.Dropoff_longitude) + 138.669195 < FLOAT(part.Dropoff_latitude),'NJ','NY') as dropOff_state FROM part").show()
taxi2DF=ss.sql("SELECT *, CONCAT(pickup_state,dropOff_state) as direction FROM (SELECT *, IF(1.323942*FLOAT(part.Pickup_longitude) + 138.669195 < FLOAT(part.Pickup_latitude),'NJ','NY') as pickup_state, IF(1.323942*FLOAT(part.Dropoff_longitude) + 138.669195 < FLOAT(part.Dropoff_latitude),'NJ','NY') as dropOff_stateunix_timestamp('Lpep_dropoff_datetime') - unix_timestamp('lpep_pickup_datetime') as duration FROM part)")
taxi2DF.createOrReplaceTempView("2DF")
#SQRT(POWER(FLOAT(part.Pickup_longitude)-FLOAT(part.Dropoff_longitude),2)+POWER(FLOAT(part.Pickup_latitude)-FLOAT(part.Dropoff_latitude),2)) as distance



ParseException: u"\nmismatched input 'FROM' expecting {<EOF>, 'WHERE', 'GROUP', 'ORDER', 'HAVING', 'LIMIT', 'LATERAL', 'WINDOW', 'UNION', 'EXCEPT', 'MINUS', 'INTERSECT', 'SORT', 'CLUSTER', 'DISTRIBUTE'}(line 1, pos 58)\n\n== SQL ==\nSELECT *, CONCAT(pickup_state,dropOff_state) as direction FROM (SELECT *, IF(1.323942*FLOAT(part.Pickup_longitude) + 138.669195 < FLOAT(part.Pickup_latitude),'NJ','NY') as pickup_state, IF(1.323942*FLOAT(part.Dropoff_longitude) + 138.669195 < FLOAT(part.Dropoff_latitude),'NJ','NY') as dropOff_stateunix_timestamp('Lpep_dropoff_datetime') - unix_timestamp('lpep_pickup_datetime') as duration FROM part)\n----------------------------------------------------------^^^\n"

In [37]:
y=40.912216186523438
x=-74.010795593261719
z=1.323942*x + 138.669195
x2=-74.010780334472656
y2=40.912212371826172
z2=1.323942*x2 + 138.669195
print(y,1.323942*x + 138.669195)
if(y>z):
    print 'NJ'
else:
    print 'NY'
if(y2>z2):
    print 'NJ'
else:
    print 'NY'
    
    

(40.91221618652344, 40.6831942606659)
NJ
NJ


In [18]:
sc.stop()
ss.stop()


In [7]:
taxi2DF.groupBy("direction").count().show()


+---------+------+
|direction| count|
+---------+------+
|     NYNJ|144110|
|     NYNY|864553|
|     NJNJ|449130|
|     NJNY| 37133|
+---------+------+



In [35]:
taxi2DF.groupby('direction').agg({'Passenger_count': "mean", 'Total_amount': "mean"}).show()

+---------+------------------+--------------------+
|direction| avg(Total_amount)|avg(Passenger_count)|
+---------+------------------+--------------------+
|     NYNJ|26.448231142874565|  1.4269516341683437|
|     NYNY| 13.73546828247377|  1.4071202112536767|
|     NJNJ|12.605316634389286|  1.2877830472246343|
|     NJNY|30.270238332479874|   1.303207389653408|
+---------+------------------+--------------------+



In [15]:
from pyspark.sql.functions import to_timestamp
taxi2DF.withColumn("lpep_pickup_datetime", to_timestamp("lpep_pickup_datetime")).show()

+--------+--------------------+---------------------+------------------+----------+-------------------+------------------+-------------------+------------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+------------+-------------+---------+
|VendorID|lpep_pickup_datetime|Lpep_dropoff_datetime|Store_and_fwd_flag|RateCodeID|   Pickup_longitude|   Pickup_latitude|  Dropoff_longitude|  Dropoff_latitude|Passenger_count|Trip_distance|Fare_amount|Extra|MTA_tax|Tip_amount|Tolls_amount|Ehail_fee|improvement_surcharge|Total_amount|Payment_type|Trip_type|pickup_state|dropOff_state|direction|
+--------+--------------------+---------------------+------------------+----------+-------------------+------------------+-------------------+------------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+-------

In [21]:
k=5

In [27]:
taxi2DF.groupBy('direction').agg({'distance': 'mean','Improvement_surcharge': 'mean'}).show()

+---------+--------------------------+--------------------+
|direction|avg(Improvement_surcharge)|       avg(distance)|
+---------+--------------------------+--------------------+
|     NYNJ|       0.29185413919932673| 0.43967106196100075|
|     NYNY|       0.29369188470782304| 0.21721722123194612|
|     NJNJ|       0.29002337853150195|0.027962606418250473|
|     NJNY|       0.28107074569787216|  1.1783874461456316|
+---------+--------------------------+--------------------+

